In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("..")

import os
import numpy as np
import pandas as pd

from src.utils import metrics, load_decompositions

datasets = sorted(["WikiText", "GitHub", "OpenWebText"])
topics = sorted([d + "_topics" for d in datasets])
models = sorted(["BERT", "GPT2", "BLOOM", "Llama2"])
index = pd.MultiIndex.from_product([models, datasets])

In [ ]:
# RUN ONLY ONCE
root_dir = "../out/decompositions/"
for dataset in datasets:
    for model in models:
        print(dataset, model)
        dataset, model = dataset.lower(), model.lower()
        out_dir = os.path.join(root_dir, f"{dataset}-{model}")
        pos, cvec, global_mean = load_decompositions.load_pos_cvec_global_mean(
            dataset, model, root_dir
        )
        results = metrics.ranks_and_explained_ratios_and_relative_norm(
            pos, cvec, global_mean
        )
        np.save(os.path.join(out_dir, "metrics.npy"), results)

root_dir = "../out/topics/"
for topic in topics:
    for model in models:
        print(topic, model)
        topic, model = topic.lower(), model.lower()
        data_dir = os.path.join(root_dir, f"{topic}-{model}")
        pos = np.load(f"{data_dir}/pos_id1-128_{model}.npy")
        L, T, C = pos.shape
        context_basis = np.memmap(
            f"{data_dir}/cbasis_id1-128_{model}.npy",
            mode="r",
            dtype=np.float32,
            shape=(L, 64 * 4, C),
        )
        B = metrics.avg_similarity_between(context_basis)
        W = metrics.avg_similarity_within(context_basis)

        np.save(f"{data_dir}/between_similarity.npy", B)
        np.save(f"{data_dir}/within_similarity.npy", W)

### Table 1 (screeNOT)

In [3]:
root_dir = "../out/decompositions/"
d = []
for model in models:
    for dataset in datasets:
        dataset, model = dataset.lower(), model.lower()
        out_dir = os.path.join(root_dir, f"{dataset}-{model}")
        results = np.load(os.path.join(out_dir, "metrics.npy"))
        c = results[:, 0]
        if len(c) > 13:
            c = c[np.linspace(0, len(c) - 1, num=13, dtype=int)]
        d.append(c)

columns = [f"Layer {i}" for i in range(13)]
pd.DataFrame(np.array(d), index=index, columns=columns).astype(int)

Layer 0  Layer 1  Layer 2  Layer 3  Layer 4  Layer 5  \
BERT   GitHub            15       16       16       16       14       11   
       OpenWebText       15       16       18       16       11       11   
       WikiText          15       16       18       16       12       11   
BLOOM  GitHub             8        9        9        8        9       10   
       OpenWebText        6       10       10       11       11       10   
       WikiText           6        8        9       10       10       11   
GPT2   GitHub            15       14       13       12       12       11   
       OpenWebText       15       13       14       12       13       11   
       WikiText          15       14       14       12       11       11   
Llama2 GitHub             6       10        9        8       10        8   
       OpenWebText        7       10       10       11       11       10   
       WikiText           8       10       10       10        9        8   

                    Layer 6  Layer 7  Layer 8  Layer 9  Layer 10  Layer 11  \
BERT   GitHub            11        9       10       10        11        11   
       OpenWebText        9        9       11       11        11        11   
       WikiText           9        9       11       11        11        12   
BLOOM  GitHub            10       11       10       10        10        10   
       OpenWebText       11       11       11       11        10        10   
       WikiText          11       11       11       11        11        10   
GPT2   GitHub            11       10       10       10        11        11   
       OpenWebText       10       10       10       10         9         9   
       WikiText          11       11       11       11         9        10   
Llama2 GitHub             8        9        9        9         9         8   
       OpenWebText        9       10        9        8         9         8   
       WikiText           8        8        8        8         8         8   

                    Layer 12  
BERT   GitHub             12  
       OpenWebText        13  
       WikiText           12  
BLOOM  GitHub             10  
       OpenWebText        11  
       WikiText           11  
GPT2   GitHub             10  
       OpenWebText        12  
       WikiText           12  
Llama2 GitHub             10  
       OpenWebText        10  
       WikiText           10

### Table 1 (stable rank)

In [4]:
root_dir = "../out/decompositions/"
d = []
for model in models:
    for dataset in datasets:
        dataset, model = dataset.lower(), model.lower()
        out_dir = os.path.join(root_dir, f"{dataset}-{model}")
        results = np.load(os.path.join(out_dir, "metrics.npy"))
        c = results[:, 1]
        if len(c) > 13:
            c = c[np.linspace(0, len(c) - 1, num=13, dtype=int)]
        d.append(c)

columns = [f"Layer {i}" for i in range(13)]
pd.DataFrame(np.array(d), index=index, columns=columns).round(2)

Layer 0  Layer 1  Layer 2  Layer 3  Layer 4  Layer 5  \
BERT   GitHub          9.19     7.79     5.26     4.73     4.34     3.84   
       OpenWebText     9.19     7.63     5.25     4.73     4.10     3.53   
       WikiText        9.19     7.78     5.03     4.58     3.99     3.48   
BLOOM  GitHub          8.39     1.25     1.20     1.21     1.21     1.23   
       OpenWebText     8.33     1.27     1.30     1.24     1.24     1.27   
       WikiText        8.42     1.27     1.28     1.30     1.31     1.34   
GPT2   GitHub          2.05     1.92     1.91     1.89     1.90     1.90   
       OpenWebText     2.05     1.92     1.91     1.89     1.88     1.88   
       WikiText        2.05     1.92     1.91     1.89     1.88     1.88   
Llama2 GitHub         24.87     1.00     1.00     1.00     1.00     1.00   
       OpenWebText    52.23     1.00     1.00     1.00     1.00     1.00   
       WikiText       24.70     1.00     1.00     1.01     1.01     1.02   

                    Layer 6  Layer 7  Layer 8  Layer 9  Layer 10  Layer 11  \
BERT   GitHub          3.48     3.20     2.70     2.45      2.04      1.84   
       OpenWebText     3.16     2.84     2.46     2.30      2.18      2.22   
       WikiText        3.14     2.82     2.42     2.27      2.13      2.16   
BLOOM  GitHub          1.29     1.29     1.28     1.25      1.21      1.02   
       OpenWebText     1.32     1.34     1.33     1.26      1.16      1.01   
       WikiText        1.41     1.43     1.41     1.32      1.22      1.01   
GPT2   GitHub          1.92     1.94     1.98     2.03      2.05      1.70   
       OpenWebText     1.88     1.90     1.91     1.96      2.02      2.24   
       WikiText        1.88     1.90     1.91     1.97      2.03      2.19   
Llama2 GitHub          1.00     1.01     1.01     1.01      1.02      1.03   
       OpenWebText     1.01     1.01     1.02     1.02      1.03      1.05   
       WikiText        1.03     1.05     1.09     1.16      1.20      1.26   

                    Layer 12  
BERT   GitHub           1.91  
       OpenWebText      2.15  
       WikiText         2.12  
BLOOM  GitHub           1.00  
       OpenWebText      1.00  
       WikiText         1.00  
GPT2   GitHub           1.11  
       OpenWebText      1.49  
       WikiText         1.56  
Llama2 GitHub           1.17  
       OpenWebText      1.44  
       WikiText         1.30

### Table 1 (Relative norm)

In [5]:
indices = pd.MultiIndex.from_product([models, datasets])
root_dir = "../out/decompositions/"
d = []
for model in models:
    for dataset in datasets:
        dataset, model = dataset.lower(), model.lower()
        out_dir = os.path.join(root_dir, f"{dataset}-{model}")
        results = np.load(os.path.join(out_dir, "metrics.npy"))
        c = results[:, 3]
        if len(c) > 13:
            c = c[np.linspace(0, len(c) - 1, num=13, dtype=int)]
        d.append(c)

columns = [f"Layer {i}" for i in range(13)]
pd.DataFrame(np.array(d), index=indices, columns=columns).round(3)

Layer 0  Layer 1  Layer 2  Layer 3  Layer 4  Layer 5  \
BERT   GitHub         0.325    0.267    0.246    0.242    0.234    0.229   
       OpenWebText    0.361    0.302    0.265    0.265    0.263    0.249   
       WikiText       0.356    0.293    0.267    0.265    0.263    0.250   
BLOOM  GitHub         0.009    0.079    0.123    0.147    0.170    0.181   
       OpenWebText    0.008    0.092    0.108    0.143    0.164    0.173   
       WikiText       0.009    0.098    0.123    0.152    0.166    0.174   
GPT2   GitHub         0.758    0.447    0.396    0.359    0.338    0.316   
       OpenWebText    0.807    0.463    0.411    0.371    0.344    0.324   
       WikiText       0.815    0.470    0.416    0.375    0.345    0.325   
Llama2 GitHub         0.025    0.221    0.221    0.221    0.222    0.222   
       OpenWebText    0.031    0.146    0.146    0.146    0.146    0.147   
       WikiText       0.035    0.067    0.067    0.067    0.067    0.068   

                    Layer 6  Layer 7  Layer 8  Layer 9  Layer 10  Layer 11  \
BERT   GitHub         0.232    0.248    0.248    0.215     0.211     0.173   
       OpenWebText    0.245    0.266    0.273    0.215     0.170     0.117   
       WikiText       0.246    0.269    0.278    0.217     0.172     0.118   
BLOOM  GitHub         0.173    0.158    0.145    0.138     0.137     0.182   
       OpenWebText    0.158    0.143    0.131    0.135     0.153     0.278   
       WikiText       0.159    0.143    0.133    0.134     0.147     0.250   
GPT2   GitHub         0.287    0.266    0.237    0.207     0.172     0.140   
       OpenWebText    0.301    0.271    0.232    0.196     0.150     0.095   
       WikiText       0.299    0.270    0.229    0.197     0.152     0.098   
Llama2 GitHub         0.222    0.223    0.224    0.225     0.226     0.226   
       OpenWebText    0.147    0.148    0.150    0.152     0.153     0.154   
       WikiText       0.068    0.069    0.071    0.073     0.077     0.080   

                    Layer 12  
BERT   GitHub          0.162  
       OpenWebText     0.122  
       WikiText        0.122  
BLOOM  GitHub          0.140  
       OpenWebText     0.385  
       WikiText        0.358  
GPT2   GitHub          0.100  
       OpenWebText     0.030  
       WikiText        0.030  
Llama2 GitHub          0.158  
       OpenWebText     0.118  
       WikiText        0.146

### Table 1 (Inter & intra similarity)

In [7]:
root_dir = "../out/topics/"
row = []
for model in models:
    for topic in topics:
        topic, model = topic.lower(), model.lower()
        data_dir = os.path.join(root_dir, f"{topic}-{model}")
        B = np.load(f"{data_dir}/between_similarity_NoPCA.npy")
        W = np.load(f"{data_dir}/within_similarity_NoPCA.npy")
        row.append([np.mean(B), np.std(B), np.mean(W), np.std(W)])

pd.DataFrame(
    np.array(row), index=index, columns=["B mean", "B std", "W mean", "W std"]
).round(2)

B mean  B std  W mean  W std
BERT   GitHub         0.17   0.04    0.31   0.05
       OpenWebText    0.13   0.04    0.26   0.04
       WikiText       0.12   0.04    0.28   0.05
BLOOM  GitHub         0.20   0.14    0.40   0.08
       OpenWebText    0.15   0.14    0.32   0.09
       WikiText       0.15   0.13    0.31   0.10
GPT2   GitHub         0.28   0.06    0.42   0.04
       OpenWebText    0.10   0.01    0.44   0.04
       WikiText       0.11   0.01    0.41   0.03
Llama2 GitHub         0.22   0.08    1.00   0.00
       OpenWebText    0.12   0.13    1.00   0.00
       WikiText       0.33   0.20    1.00   0.00